<a href="https://colab.research.google.com/github/arahrooh31/UCLA_BE223C/blob/Keane_temp/VGG_DBT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import gc  #debug memory leaks in matplotlib
import csv #read in description files

from google.colab.patches import cv2_imshow
#
# Read Data from google drive
#

from google.colab import drive #for loading gdrive data
from google.colab import files

# install dependencies not included by Colab
# use pip3 to ensure compatibility w/ Google Deep Learning Images 
!pip3 install -q pydicom 
!pip3 install -q tqdm 
!pip3 install -q imgaug
!pip3 install -q pickle5

import pydicom #to read dicom files
from pydicom import dcmread
import pickle5 as pickle; #generic storage of image arra


import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
#show model design parameters with torchsummary
import torchsummary
from torchsummary import summary
from torch import FloatTensor
from torch import tensor


### Enable GPU, if present
train_on_gpu = torch.cuda.is_available()
if (train_on_gpu):
    !nvidia-smi -L
    !nvidia-smi 
    dev=torch.device("cuda")
else:
    print('GPU NOT FOUND!!! USING CPU INSTEAD!!!!!')

train_on_tpu =0 #enable TPUs
if ((train_on_gpu == 0 ) and train_on_tpu == 1):
    !pip3 install -q cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8.1-cp37-cp37m-linux_x86_64.whl
    # imports the torch_xla package
    import torch_xla
    import torch_xla.core.xla_model as xm
    dev = xm.xla_device()
    print('!!!! USING TPU!!!!')
#
# Load data from google drive
#

drive.mount('/content/gdrive')

data_dir = '/content/gdrive/My Drive/DBT_DATA/IMG_ARRAYS'
patch_dir = '/content/gdrive/My Drive/DBT_WORKSPACE/TRAINING_PATCHES' 
model_dir = '/content/gdrive/My Drive/BE223C_SPRING_2021/MODEL_SAVE'
tensorboard_dir = '/content/gdrive/My Drive/BE223C_SPRING_2021/TENSORBOARD_SUMMARIES'
#DBT_DATA/TRAINING_DATA/manifest-1605042674814/Breast-Cancer-Screening-DBT'
#png_dir = '/content/gdrive/My Drive/BE223C_SPRING_2021/PNG_IMAGES'
#png_annotation_dir = '/content/gdrive/My Drive/BE223C_SPRING_2021/PNG_ANNOTATION_IMAGES'
#image_save_dir = '/content/gdrive/My Drive/DBT_DATA/IMG_ARRAYS' #matrix img data
#'/content/gdrive/My Drive/BE223C_SPRING_2021/IMAGE_ARRAY'


GPU NOT FOUND!!! USING CPU INSTEAD!!!!!
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
#
# GET FULL LIST OF FILES IN IMAGE ARRAY DIRECTORY
#
use_patch_files = 1
if (use_patch_files == 0):
    raw_files = os.listdir(data_dir)
    print('found #files: ',len(raw_files))
else: #patches broken up into directories
    category_folders = os.listdir(patch_dir)
    #raw_files = os.listdir(data_dir)
    #print('found #files: ',len(raw_files))    

if (0):
    #create fake patches for now
    patch_dict = {}
    for counter,filename in enumerate(raw_files):
        #load full array
        full_filename = os.path.join(data_dir,filename)
        img_data = pickle.load( open( full_filename, "rb" ) )
        patch_data = img_data[0:3,:,:]
        patch_dict[counter]= patch_data
        print(full_filename,np.shape(patch_data))

        if (counter > 0):
            break
        
### SKIP ACTIONABLE FOLDER
### Remove actionable folder item from list, since we're no longer using that data
category_folders.remove('ACTIONABLE')

In [3]:
for ii in category_folders:
    print(ii)
    flist = os.listdir(os.path.join(patch_dir,ii))

temp = flist[0].split(sep='_')
print(temp[3])

CANCER
BENIGN
NORMAL
Normal


In [4]:
#generate full file list for use in indexing the dataloader
#this replaces the older loader, which was only inputting Normal
full_file_list = [] #store the full filename of every file
full_category_name = []
for category_folder in category_folders:
            print('------------- ',category_folder)
            file_list = os.listdir(os.path.join(patch_dir,category_folder))
            cat_count = 0
            for file_name in file_list:
                cat_count = cat_count + 1
                full_category_name.append(category_folder)
                full_file_list.append(file_name)
                print(os.path.join(category_folder,file_name))
full_file_count = len(full_file_list)
print(len(full_file_list))
full_file_list[0]


-------------  CANCER
CANCER/DBT-P03915_DBT-S05004_lcc_Cancer_s25_cx417_cy1039_244_244.pickle
CANCER/DBT-P04026_DBT-S01650_rcc1_Cancer_s20_cx1640_cy1074_244_244.pickle
CANCER/DBT-P04026_DBT-S01650_rmlo_Cancer_s18_cx1532_cy1382_244_244.pickle
CANCER/DBT-P04090_DBT-S01718_rmlo_Cancer_s59_cx1212_cy825_244_244.pickle
CANCER/DBT-P04090_DBT-S01718_rcc_Cancer_s63_cx1368_cy1342_244_244.pickle
CANCER/DBT-P04372_DBT-S04281_lcc_Cancer_s34_cx963_cy1301_244_244.pickle
CANCER/DBT-P04372_DBT-S04281_lmlo_Cancer_s39_cx1197_cy1391_244_244.pickle
CANCER/DBT-P04631_DBT-S05515_lcc_Cancer_s9_cx546_cy1059_244_244.pickle
CANCER/DBT-P04631_DBT-S05515_lmlo_Cancer_s18_cx692_cy1761_244_244.pickle
CANCER/DBT-P04710_DBT-S03227_rmlo_Cancer_s42_cx1467_cy1509_244_244.pickle
CANCER/DBT-P04710_DBT-S03227_rcc_Cancer_s19_cx1420_cy1431_244_244.pickle
CANCER/DBT-P04901_DBT-S05032_rcc_Cancer_s35_cx1534_cy961_244_244.pickle
CANCER/DBT-P04901_DBT-S05032_rmlo_Cancer_s35_cx1252_cy1064_244_244.pickle
CANCER/DBT-P05030_DBT-S05569_

'DBT-P03915_DBT-S05004_lcc_Cancer_s25_cx417_cy1039_244_244.pickle'

In [5]:
import os
import pandas as pd
from torchvision.io import read_image
from torchvision import transforms  #get normalization functions

class CustomImageDataset(): #Dataset):
    def __init__(self, img_dir,category=[],file_count=1,file_list =[],transform=None, target_transform=None):
        #self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.category = category
        self.file_count = file_count
        self.file_list = file_list
        self.transform = transform
        self.target_transform = target_transform
        self.category_name =''
        



    def image_normalize(image):
        #replace with the more tensor friendly normalize once tensor shapes confirmed
        image = image/65535.0
        return image


    def __len__(self):
        return self.file_count #len(self.file_list) #99 #len(self.img_labels)

    def __getitem__(self, index):
        
        fname = self.file_list[index]

        #get label and pull category 
        text_tokens = fname.split(sep='_')
        label_class = text_tokens[3] #get the label token in 4th position
        self.category_name =  label_class.upper() 


        full_file_name = os.path.join(self.img_dir,self.category_name,fname)
        image = pickle.load( open( full_file_name, "rb" ) )
        image = image.astype(float) #using patch images
        

        #VERIFY THE IMAGES ARE ALL THE SAME 3x244x244
        shapes = image.shape
        assert (shapes[0] == 3),"Image slice error: {0}".format(fname)
        assert (shapes[1] == 244), print('Image row error: ',fname)
        assert (shapes[2] == 244), print('Image column error: ',fname)
        #if (shapes[0] != 3 and shapes[1] != 244 and shapes[2]!= 244):


        #Normalize the data to 0,1 from 2^16
        image = image/65535.0 #image_normalize(image)



        #test out numeric label
        if (label_class in 'Normal'):
            label = 0
        elif (label_class in 'Actionable'):
            print('!!!!! ACTIONABLE PASSED THROUGH')
            stop()
            label = 1
        elif (label_class in 'Benign'):
            label = 1
        else: # (label_class in 'Cancer'):
            label = 2


        #print(full_file_name)


        #read_image(img_path)
        #label = self.img_labels.iloc[idx, 1]
        #if self.transform:
        #    image = self.transform(image)
        #if self.target_transform:
        #    label = self.target_transform(label)
        sample = {"image": image, "label": label}
            #sample = file_name
        return sample


        

In [6]:
#
# Adjust the weighting and sampling of the training data to correct for any
#imbalances. This is used in loss and dataloading
#

#GENERATE THE WEIGHT TENSOR BASED UPON THE LABEL APPLIED
weight_list =[]
for ii in  full_file_list:
        
    for jj in ii:
        if ('Normal' in jj):
            wval = 1
        elif ('Benign' in jj):
            wval = 3
        else:
            wval = 5

        weight_list.append(wval)

    #count = count + len(nfiles)
number_weight_values = len(weight_list)
number_weight_values = torch.tensor(number_weight_values)
print('Number of weight values = ',number_weight_values)

#target_list = full_file_list[torch.randperm(len(full_file_list))]
weight_numbers = torch.tensor([1,15,27], dtype=torch.float32)

#class_weights = [num_samples/class_counts[i] for i in range(len(class_counts))]
#weights = [class_weights[labels[i]] for i in range(int(num_samples))]
sampler = torch.utils.data.WeightedRandomSampler(torch.DoubleTensor(weight_list), int(4))


#sampler = torch.utils.data.WeightedRandomSampler(weights=weight_numbers, 
#                                                 num_samples=number_weight_values, replacement=True)

Number of weight values =  tensor(120306)


In [7]:
from torch.utils.data import DataLoader

#
count =0
# Setup the custom dataset
for ii in category_folders:
        nfiles = os.listdir(os.path.join(patch_dir,ii))
        count = count + len(nfiles)
print('Number of patch files found = ',count)

#load up with the pre-sized patch images
training_data = CustomImageDataset(img_dir=patch_dir,
                                   category = full_category_name, 
                                   file_count=full_file_count,
                                   file_list = full_file_list, 
                                   transform=None, 
                                   target_transform=None)

#
# TEST OUT SPLITTING DATASETS INTO TRAIN/TEST
#
train_size = int(0.8 * len(training_data))
val_size = int(0.1 * len(training_data))
test_size = len(training_data) - train_size - val_size
train_subset, val_subset, test_subset = torch.utils.data.random_split(training_data, [train_size, val_size,test_size])


bsize = 50
#training_data = CustomImageDataset( annotations_file='', img_dir=data_dir, file_list=raw_files,transform=None, target_transform=None)
if (train_on_gpu==1):
    print('setting GPU data loads')
    dataloader_training = DataLoader(train_subset, batch_size=bsize,shuffle=True, num_workers=2) #only 2 workers for Colab CPU
    dataloader_validation = DataLoader(val_subset, batch_size=bsize,shuffle=True, num_workers=2) #only 2 workers for Colab CPU
    dataloader_test = DataLoader(test_subset, batch_size=bsize,shuffle=True, num_workers=2) #only 2 workers for Colab CPU
elif ((train_on_tpu == 1) and (train_on_gpu == 0)):
    print('setting TPU data loads')
    dataloader_training = DataLoader(train_subset, batch_size=bsize,shuffle=True, num_workers=2) #only 2 workers for Colab CPU
    dataloader_validation = DataLoader(val_subset, batch_size=bsize,shuffle=True, num_workers=2) #only 2 workers for Colab CPU
    dataloader_test = DataLoader(test_subset, batch_size=bsize,shuffle=True, num_workers=2) #only 2 workers for Colab CPU

else:
    dataloader_training = DataLoader(train_subset, batch_size=bsize,shuffle=True, num_workers=2)#, sampler=sampler) #only 2 workers for Colab CPU
    dataloader_validation = DataLoader(val_subset, batch_size=bsize,shuffle=True, num_workers=2,drop_last=True)#, sampler=sampler) #only 2 workers for Colab CPU
    dataloader_test = DataLoader(test_subset, batch_size=bsize,shuffle=True, num_workers=2) #only 2 workers for Colab CPU


############################################
if (0): #disable while testing above, but this works
    bsize = 8
    #training_data = CustomImageDataset( annotations_file='', img_dir=data_dir, file_list=raw_files,transform=None, target_transform=None)
    if (train_on_gpu):
        dataloader = DataLoader(training_data, batch_size=bsize,shuffle=True, num_workers=4) #only 2 workers for Colab CPU
    else:
        dataloader = DataLoader(training_data, batch_size=bsize,shuffle=True, num_workers=2) #only 2 workers for Colab CPU


Number of patch files found =  1859


In [8]:
### DEBUG
#for ii in training_data:
#    print(ii)
#print(full_file_list[401])
#d = next(iter(dataloader_training))
#print(len(d['image']), len(d['label']))
#print(d['label'])
if (0):
    for i, _ in enumerate(dataloader_training, 0):
        # get the inputs; data is a list of [inputs, labels]
        #inputs, labels = data
        pass
        #print(type(data['image']))


        if (i > 0):
            break

if (0):
    #Mixing GPU and CPU model saves doesn't seem to map well yet
    final_file = os.path.join(model_dir,'vgg16_best_accuracy_97_gpu')
    checkpoint = torch.load(final_file, map_location=torch.device('cpu'))
    #model_vgg16 = VGG16(*args, **kwargs)
    #model_vgg16.load_state_dict(torch.load(PATH))
    
    #model_vgg16.load_state_dict(checkpoint)
    #best_acc = checkpoint['acc']
    #start_epoch = checkpoint['epoch']

In [8]:
from torch import nn

class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.flatten = nn.Flatten()
        self.vgg16_stack = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),  
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),         
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),        
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),            
            nn.ReLU(inplace=True),         
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),         
            nn.MaxPool2d(kernel_size=2, stride=2),

            #flattening layer before the linear??
            nn.Flatten(), #testing this out before FC layers
            nn.Linear(25088, 4096), #in should match 512x512 above
            nn.ReLU(inplace=True), #testing this layer out instead of softmax
            nn.Linear(4096,3))#, #3 classes instead of 4, removed Actionable
            #nn.Softmax(dim=1))
        #nn.Sequential(nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)

        #self.linear_layers = Sequential(
        #    Linear(4 * 7 * 7, )
        #)

#transform_test = transforms.Compose([
#    transforms.ToTensor(),
#    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
#])

    def forward(self, x):
        #self.flatten = nn.Flatten()
        #x = self.flatten(x)
        #print('fwd shape x = ',x.shape)
        logits = self.vgg16_stack(x)
        #print('logits out = ', logits.shape)
        
        return logits



model_vgg16 = VGG16() #.to(device)
model_vgg16 = model_vgg16.float()
print(model_vgg16)

VGG16(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (vgg16_stack): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Conv2d(256, 512, kernel_size=(3, 3),

In [10]:
#
# GET TPU Information  --- use only when GPU has timed out
# EXPERIMENT!!!!
if (train_on_tpu == 1):
    import tensorflow as tf
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
    tf.config.experimental_connect_to_cluster(resolver)
    # This is the TPU initialization code that has to be at the beginning.
    tf.tpu.experimental.initialize_tpu_system(resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))

In [11]:
#clear GPU cache if needed

try_to_clear = 0
if (try_to_clear == 1):
    del model_vgg16
    mem_alloc = torch.cuda.memory_allocated(dev)
    gc.collect()
    torch.cuda.empty_cache() #   clear_cache

    print('memory allocated is ', mem_alloc)
else:
    print('SKIP GPU MEM CLEAR---')

SKIP GPU MEM CLEAR---


In [9]:
#Show summary of model setup and move model to the GPU
 #train_on_gpu = torch.cuda.is_available()

if (train_on_gpu == 1):
    #dev=torch.device("cuda") 
    model_vgg16.to(dev)
    summary(model_vgg16,(3,244,244), batch_size = bsize, device='cuda')
elif ( (train_on_tpu == 1) and (train_on_gpu == 0)):
    ### TPU with pytorch has some issues
    model_vgg16.to(dev)
    summary(model_vgg16,(3,244,244), batch_size = bsize, device=dev)
else:
    summary(model_vgg16,(3,244,244), batch_size = bsize)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [50, 64, 244, 244]           1,792
            Conv2d-2         [50, 64, 244, 244]          36,928
              ReLU-3         [50, 64, 244, 244]               0
         MaxPool2d-4         [50, 64, 122, 122]               0
            Conv2d-5        [50, 128, 122, 122]          73,856
            Conv2d-6        [50, 128, 122, 122]         147,584
              ReLU-7        [50, 128, 122, 122]               0
         MaxPool2d-8          [50, 128, 61, 61]               0
            Conv2d-9          [50, 256, 61, 61]         295,168
           Conv2d-10          [50, 256, 61, 61]         590,080
           Conv2d-11          [50, 256, 61, 61]         590,080
             ReLU-12          [50, 256, 61, 61]               0
        MaxPool2d-13          [50, 256, 30, 30]               0
           Conv2d-14          [50, 512,

In [10]:
#
# LOSS FUNCTION SETUP
#
import torch.optim as optim
from torchsummary import summary

import torch
torch.autograd.set_detect_anomaly(True)

#criterion = nn.CrossEntropyLoss()

criterion = nn.CrossEntropyLoss(weight = weight_numbers)
if (train_on_gpu ==1):
    criterion.cuda(dev)

#OPTIMIZERS
#optimizer = optim.SGD(model_vgg16.parameters(), lr=0.1, momentum = 0.99) #lr=0.001, momentum=0.9)
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

optimizer = torch.optim.Adam(model_vgg16.parameters(), lr=0.000007, weight_decay=0.00000)#lr=0.00005, weight_decay=0.0000)

#add scheduler to adjust learning rates when val gets stuck
#scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.1)

if (train_on_gpu == 1):
    m = nn.LogSoftmax(dim=1).cuda(dev)
    nll_loss = nn.NLLLoss().cuda(dev)
else:
    m = nn.LogSoftmax(dim=1)
    nll_loss = nn.NLLLoss()

L1loss = nn.L1Loss()

model_vgg16.parameters
#summary(model_vgg16, (3, 224, 224))

<bound method Module.parameters of VGG16(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (vgg16_stack): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): C

In [11]:
#tensorboard for debugging views
use_tensor_board = 1
if (use_tensor_board == 1):
    from torch.utils.tensorboard import SummaryWriter
    writer = SummaryWriter(tensorboard_dir)

In [15]:
#Look at Tensorboard info
if (0):
    !pip3 install -q tensorboard
    %tensorboard --logdir=tensorboard_dir

In [16]:
###Generate Data weight values
#weight_numbers = torch.tensor([1,40,80], dtype=torch.float32)
#[1600,124,76] [1,13,21]
#weight_sum = torch.sum(weight_numbers)
##weight_balance=[]
#for ii in weight_numbers:
#    weight_balance.append(ii/weight_sum)
#torch.tensor(weight_balance)

#newList = [x / sum(weight_sum) for x in weight_balance]

In [ ]:
##########################################################
# Test against the test dataset to get accuracy results
##########################################################
run_test= 1
if (run_test == 1):
    #Mixing GPU and CPU model saves doesn't seem to map well yet
    final_file = os.path.join(model_dir,'vgg16_best_accuracy_97_gpu_final')
    checkpoint = torch.load(final_file, map_location=torch.device('cpu'))
    #model_vgg16 = VGG16(*args, **kwargs)
    #model_vgg16.load_state_dict(torch.load(PATH))
    
    #model_vgg16.load_state_dict(checkpoint)
    #best_acc = checkpoint['acc']
    #start_epoch = checkpoint['epoch']



    model_vgg16.load_state_dict(checkpoint)
    model_vgg16.eval()


    #load up with the pre-sized patch images
    all_data = CustomImageDataset(img_dir=patch_dir,
                                    category = full_category_name, 
                                    file_count=full_file_count,
                                    file_list = full_file_list, 
                                    transform=None, 
                                    target_transform=None)

    dataloader_all = DataLoader(all_data, batch_size=50,shuffle=True, num_workers=2)#, 

    total_accuracy = []
    total_precision = []
    for epoch in range(0,35):
        with torch.no_grad():
            for i, data in enumerate(dataloader_all, 0):
                print(i)
                # get the inputs; data is a list of [inputs, labels]
                #inputs, labels = data
                inputs = data['image'].type(FloatTensor)
                labels = data['label'] #.type(FloatTensor)

                if (train_on_gpu):
                    inputs, labels = inputs.to(dev), labels.to(dev)

                # forward + backward + optimize
                outputs = model_vgg16(inputs) #.permute(0, 1, 2, 3))

                outputs=torch.flatten(outputs, start_dim=1)
                loss = criterion(outputs, labels.long())


                #print(outputs)
                y_pred_softmax = torch.log_softmax(outputs, dim = 1)
                _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
                #print(y_pred_tags)
                #print(labels)
                correct_pred = (y_pred_tags == labels).float()
                accuracy = correct_pred.sum() / len(correct_pred)
                #accuracy = torch.round(accuracy)

                #calculate PRECISION
                #get # of true cancer cases, find # of cancer cases falsely predicted
                from sklearn.metrics import average_precision_score
                labels 
                temp_labels = []
                temp_prediction = []
                for ii in labels:
                    if (ii == 2):
                        temp_labels.append(1)
                    else:
                        temp_labels.append(0)

                for ii in y_pred_tags:
                    if (ii == 2):
                        temp_prediction.append(1)
                    else:
                        temp_prediction.append(0)
                print(temp_labels)
                print(temp_prediction)                   
                average_precision = average_precision_score(temp_labels, temp_prediction)
                total_precision.append(average_precision)
                print('Average precision-recall score: {0:0.2f}'.format(
                    average_precision))


                total_accuracy.append(accuracy)
                if (i%100 == 0):
                    print('@ interim accuracy = ',i,  sum(total_accuracy)/len(total_accuracy))                
                #print('-----#correct, training accuracy = ',correct_pred,accuracy)
    print('Finished testing all data')
    print('total accuracy = ', sum(total_accuracy)/len(total_accuracy))

0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Average precision-recall score: nan
@ interim accuracy =  0 tensor(1.)
1


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_ranking.py:677: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Average precision-recall score: 0.67
2
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Average precision-recall score: 1.00
3
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Average precisio

In [19]:
temp_labels = []
for ii in labels:
    if (ii == 2):
        temp_labels.append(1)
    else:
        temp_labels.append(0)
print(labels)
print(temp_labels)

tensor([0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0])
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
### TRAIN THE NETWORK
from torch import FloatTensor
from torch import tensor


start_epoch = 0 #used for reloading model
stop_epoch = 100
training_loss = {}
validation_loss = {}
training_accuracy = {}
tloss = []
vloss = []
best_accuracy = 0.0 #store highest accuracy for current model

##################################################################
############# RESUME FROM EARLIER CHECKPOINT
###################################################################
resume_checkpoint = 0
if (resume_checkpoint == 1):
    #Mixing GPU and CPU model saves doesn't seem to map well yet
    check_file = os.path.join(model_dir,'model_final_1')
    checkpoint = torch.load(check_file, map_location=torch.device('cpu'))
    model_vgg16.load_state_dict(checkpoint)
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']
    

for epoch in range(start_epoch,stop_epoch): #(2):  # loop over the dataset multiple times

    training_loss[epoch] = []
    validation_loss[epoch] = []
    training_accuracy[epoch] = []

#
# TRAINING SECTION
#
    model_vgg16.train()     # Optional when not using Model Specific layer
    running_loss = 0.0
    iteration_counter = 0 #keep track of total runs
    #for i, data in enumerate(dataloader, 0):  #works for general setup!!
    for i, data in enumerate(dataloader_training, 0):
        model_vgg16.train() #reset if validation step kicked off
        iteration_counter = iteration_counter + 1
        # get the inputs; data is a list of [inputs, labels]
        #inputs, labels = data
        inputs = data['image'].type(FloatTensor)
        labels = data['label'] #.type(FloatTensor)

        if (train_on_gpu):
            inputs, labels = inputs.to(dev), labels.to(dev)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model_vgg16(inputs) #.permute(0, 1, 2, 3))
        if (i == 0):
            #show the initial sizes as a check
            print(outputs.size())
            print(labels.size())

        ### L1 loss complains about outputs columns being out of order, permute
        ### outputs to match up
        use_L1 = 0
        if (use_L1==1):
            outputs= outputs.permute(3,1,2,0)
            loss = L1loss(outputs,labels)
        else:
            #outputs = nn.Softmax2d(outputs)
            
            outputs=torch.flatten(outputs, start_dim=1)
            #weight_balance = torch.tensor([0,0])
            #loss = criterion(outputs, labels.long())
            loss = criterion(outputs, labels.long())
        training_loss[epoch].append(loss.item())
        print('Loss = ', loss.item())

        #
        # ACCURACY
        #
        if (i%2 == 0):
            #print(outputs)
            y_pred_softmax = torch.log_softmax(outputs, dim = 1)
            _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
            print(y_pred_tags)
            print(labels)
            correct_pred = (y_pred_tags == labels).float()
            accuracy = correct_pred.sum() / len(correct_pred)
            #accuracy = torch.round(accuracy)
            training_accuracy[epoch].append(accuracy)
            print('-----#correct, training accuracy = ',correct_pred,accuracy)

        #
        # TENSOR BOARD ADD
        #
        if (use_tensor_board == 1):
            writer.add_scalar("Loss/train", loss, epoch)


        loss.backward()
        optimizer.step()

        # print statistics
        if (i %50 == 0):
            if(train_on_gpu == 1):
                !nvidia-smi
        if (i % 100 == 0):
            running_loss += loss.item()
            print('loop,epoch, loss, total running loss = ',i,epoch, loss.item(), running_loss/100)
            running_loss = 0.0


###############################################
# VALIDATION STEP
###############################################
    if (iteration_counter %1 == 0):
        model_vgg16.eval()     # Optional when not using Model Specific layer
        valid_loss = 0.0

        with torch.no_grad():
            for ii, vdata in enumerate(dataloader_validation, 0):
                # get the inputs; data is a list of [inputs, labels]
                #inputs, labels = data
                vinputs = vdata['image'].type(FloatTensor)
                vlabels = vdata['label'] #.type(FloatTensor)

                #assert(len(vlabels) == bsize), "Wrong number of val labels"

                if (train_on_gpu):
                    vinputs, vlabels = vinputs.to(dev), vlabels.to(dev)

                target = model_vgg16(vinputs)

                #flatten for cross entropy loss
                target=torch.flatten(target, start_dim=1)
                loss = criterion(target,vlabels)
                valid_loss = loss.item()# * data.size(0)
                validation_loss[epoch].append(valid_loss)


                y_val_softmax = torch.log_softmax(target, dim = 1)
                _, y_val_tags = torch.max(y_val_softmax, dim = 1)
                print(y_val_tags)
                print(vlabels)
                correct_pred = (y_val_tags == vlabels).float()
                val_accuracy = correct_pred.sum() / len(correct_pred)
                #accuracy = torch.round(accuracy)
                #training_accuracy[epoch].append(accuracy)
                print('....#correct, validation accuracy = ',correct_pred,val_accuracy)
                if ((val_accuracy > best_accuracy) and (valid_loss < 0.5) and (valid_loss > 0.05)):
                    print('!!!!!Accuracy < previous. Saving model. acc= ',val_accuracy)
                    best_accuracy = val_accuracy
                    #save this model off
                    best_model_name = 'vgg16_best_accuracy_' + str(np.int(best_accuracy.item()*100))
                    torch.save(model_vgg16.state_dict(),os.path.join(model_dir,best_model_name))





####################################################
# SAVE INTERMEDIATE FILES
####################################################

    training_pickle = os.path.join(tensorboard_dir,'tensor_training_loss.pickle')
    validation_pickle = os.path.join(tensorboard_dir,'tensor_validation_loss.pickle')
    accuracy_pickle = os.path.join(tensorboard_dir,'tensor_training_accuracy.pickle')
    val_accuracy_pickle = os.path.join(tensorboard_dir,'tensor_validation_accuracy.pickle')

    pickle.dump(training_loss, open( training_pickle, "wb" ),protocol=5 )
    pickle.dump(validation_loss, open( validation_pickle, "wb" ),protocol=5 )
    pickle.dump(training_accuracy,open( accuracy_pickle, "wb" ),protocol=5 )
    pickle.dump(val_accuracy,open( val_accuracy_pickle, "wb" ),protocol=5 )


    #save a midterm file
    if (1): #epoch %2 == 0):
        save_model_mid = 'model_final_' + str(epoch)
        torch.save({
                    'epoch': epoch,
                    'model_state_dict': model_vgg16.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss,
                    }, os.path.join(model_dir,save_model_mid))

    if (use_tensor_board == 1):
        writer.flush() #write out all the info to the board

torch.save(model_vgg16.state_dict(),os.path.join(model_dir,'vgg16_trained_final'))
print('Finished Training')
if (use_tensor_board == 1):
    writer.close()

torch.Size([50, 3])
torch.Size([50])
Loss =  1.0991638898849487
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0])
-----#correct, training accuracy =  tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.]) tensor(0.9200)
loop,epoch, loss, total running loss =  0 0 1.0991638898849487 0.010991638898849488
Loss =  1.095999002456665
Loss =  1.098305344581604
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2])

In [ ]:
#### Test out saved pickle files
#tdata = training_pickle
training_pickle = os.path.join(tensorboard_dir,'tensor_training.pickle')
validation_pickle = os.path.join(tensorboard_dir,'tensor_validation.pickle')
accuracy_pickle = os.path.join(tensorboard_dir,'tensor_accuracy.pickle')

tdata = pickle.load( open( training_pickle, "rb" ) )
vdata =  pickle.load(open(validation_pickle,"rb"))
adata =  pickle.load(open(accuracy_pickle,"rb"))
#torch.load(accuracy_pickle,map_location=torch.device('cpu'))

tdata.keys()

total_losses= []
for ii in tdata.keys():
    for num in tdata[ii]:
        total_losses.append(num)

plt.figure()
plt.plot(total_losses)

total_validation = []
for ii in vdata.keys():
    for num in vdata[ii]:
        total_validation.append(num)

plt.figure()
plt.plot(total_validation)


total_accuracy = []
for ii in adata.keys():
    for num in adata[ii]:
        total_accuracy.append(num)


plt.figure()
plt.plot(total_accuracy)




In [ ]:
#
#Disconnect from the VM to save our GPU time
#
!kill -9 -1

In [ ]:
model_weights = [] # we will save the conv layer weights in this list
conv_layers = [] # we will save the 49 conv layers in this list
model_children = list(model_vgg16.children())
print(len(model_children))
print(model_children[1])


# counter to keep count of the conv layers
counter = 0 
# append all the conv layers and their respective weights to the list
for i in range(len(model_children[1])):
    if type(model_children[1][i]) == nn.Conv2d:
        counter += 1
        model_weights.append(model_children[1][i].weight)
        conv_layers.append(model_children[1][i])
    elif type(model_children[1][i]) == nn.Sequential:
        for j in range(len(model_children[1][i])):
            for child in model_children[1][i][j].children():
                if type(child) == nn.Conv2d:
                    counter += 1
                    model_weights.append(child.weight)
                    conv_layers.append(child)
print(f"Total convolutional layers: {counter}")
print(len(model_weights))
print(np.shape(model_weights[0]))

# take a look at the conv layers and the respective weights
for weight, conv in zip(model_weights, conv_layers):
    # print(f"WEIGHT: {weight} \nSHAPE: {weight.shape}")
    print(f"CONV: {conv} ====> SHAPE: {weight.shape}")




# visualize the first conv layer filters
print(np.shape(model_weights[3]))
plt.figure(figsize=(20, 17))
for i, filter in enumerate(model_weights[0]):
    plt.subplot(8, 8, i+1) # (8, 8) because in conv0 we have 7x7 filters and total of 64 (see printed shapes)
    plt.imshow(filter[0, :, :].detach(), cmap='gray')
    plt.axis('off')
    #plt.savefig('../outputs/filter.png')
plt.show()

In [ ]:
check_file = os.path.join(model_dir,'model_final_15')
checkpoint = torch.load(check_file)#, map_location=torch.device('cpu'))
#model_vgg16.load_state_dict(checkpoint)
#best_acc = checkpoint['acc']
#start_epoch = checkpoint['epoch']
model_vgg16.eval()

In [ ]:
outputs #.softmax(dim=1)
loss
y_pred_softmax = torch.log_softmax(outputs, dim = 1)
_, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
print(y_pred_tags)
print(labels)
correct_pred = (y_pred_tags == labels).float()
acc = correct_pred.sum() / len(correct_pred)

acc = torch.round(acc * 100)
print('correct_pred, accuracy = ',correct_pred,acc)
#torch.flatten(outputs, start_dim=1)

In [ ]:
#target.topk
#target = model_vgg16(vinputs)
target = outputs
print(inputs.shape)
#vout = model_vgg16(inputs)
print(vout.shape)
vout_save = vout

#probabilities = torch.exp(vout).data

# getting the topk (=5) probabilites and indexes
# 0 -> probabilities
# 1 -> index
print('vout size = ', vout.shape)
#probs = torch.log_softmax(target, dim=1)
#probs = torch.softmax(vout.flatten, dim=1)
probs = torch.flatten(vout,3)
#print('prob shape = ',probs.shape)
print(torch.softmax(probs,dim=0))
#prob = torch.topk(probabilities, topk)[0].tolist()[0] # probabilities
#index = torch.topk(probabilities, topk)[1].tolist()[0] # index
print(probs[0,:].sum()/len(probs))

In [ ]:
import torch.nn.functional as nnf

# ...
prob = nnf.softmax(outputs, dim=1)

top_p, top_class = prob.topk(1, dim = 1)
print(top_p,top_class)

In [ ]:
counter = 0
for ii in dataloader_validation:
    print(ii)
    counter= counter + 1
    if (counter > 0):
        break

In [ ]:
#outputs=torch.flatten(outputs, start_dim=1)
#loss = criterion(outputs, labels.long())

outputs = model_vgg16(inputs) #.permute(0, 1, 2, 3))
print(outputs.size())
print(labels.size())
a= torch.flatten(outputs,start_dim=1)

print(outputs.mean())
print(a)
criterion(a,labels)

In [ ]:
#model_vgg16.state_dict()
print(outputs.shape)
a=outputs.view(4*512*7,4)
print(a.shape)
print(labels)
x=torch.flatten(outputs, start_dim=1)

#proper_target = torch.argmax(labels, dim=1)  # make sure keepdim=False
#loss = criterion(outputs, proper_target)


In [ ]:
torch.tensor([2])
a=outputs/2
a = torch.tensor(a)
m = np.mean(inputs)
b= inputs/1000
b

In [ ]:
num_correct = 0
total = 0
model_vgg16.eval()

with torch.no_grad():
    for i, data in enumerate(dataloader_test, 0):
    #for data, labels in test_subset:
        #data = data.to(device=device)
        #labels = labels.to(device=device)
        inputs = data['image'].type(FloatTensor)
        labels = data['label'] #.type(FloatTensor)

        predictions = model_vgg16(inputs) #for testing, only pass in images, no labels
        predictions = predictions.permute(3,1,2,0)
        num_correct += (predictions == labels).sum()
        total += labels.size(0)

    print(f"Test Accuracy of the model: {float(num_correct)/float(total)*100:.2f}")

In [ ]:
#
# Test out every file for the correct shape
#
total_files = len(full_file_list)
for counter, file_name in enumerate(full_file_list):

    text_tokens = file_name.split(sep='_')
    label_class = text_tokens[3] #get the label token in 4th position
    category_name =  label_class.upper() 

    #print('testing file = ', counter, file_name)
    cfile = os.path.join(patch_dir,category_name,file_name)
    image = pickle.load( open( cfile, "rb" ) )
    #print('testing ',cfile)
    shape = np.shape(image)
    if ((shape[0] != 3) or (shape[1] != 244) or (shape[2] != 244)):
        print('Failed file ',file_name, shape)

#    else:
#        print('testing file = ', counter, file_name, shape)



In [ ]:
#patch_dir = '/content/gdrive/My Drive/DBT_WORKSPACE/TRAINING_PATCHES' 
!ls /content/gdrive/MyDrive/DBT_WORKSPACE/TRAINING_PATCHES/NORMAL

In [ ]:

cfile = os.path.join(patch_dir,'NORMAL','DBT-P03406_DBT-S05192_lcc_Normal_s83_cx520_cy1165_244_244.pickle')
image = pickle.load( open( cfile, "rb" ) )
image.shape
